In [5]:
import pandas as pd
import numpy as np
from math import sqrt
import time

import numba
from numba import int32, float64
from numba.experimental import jitclass

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
#https://stackoverflow.com/questions/21154643/python-line-profiler-installation
%load_ext line_profiler

In [7]:
digits = load_digits()
print(digits.data.shape)


#image representatio of the data
# plt.gray() 
# plt.matshow(digits.images[0]) 
# plt.show() 


X = digits.data 
y = digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

(1797, 64)


In [13]:
len(X_train),len(X_test)

(1347, 450)

In [8]:
class KNN:
    def __init__(self, K=3):
        self.K = K

    def fit(self, x_train, y_train):
        self.X_train = x_train
        self.Y_train = y_train

    
    def euc_dist(self, x1, x2):
    	return np.sqrt(np.sum((x1-x2)**2))
    	
    
    def predict(self, X_test):
	    predictions = [] 
	    for i in range(len(X_test)):
	        dist = np.array([self.euc_dist(X_test[i], x_t) for x_t in self.X_train])
	        dist_sorted = dist.argsort()[:self.K]
	        neigh_count = {}
	        for idx in dist_sorted:
	            if self.Y_train[idx] in neigh_count:
	                neigh_count[self.Y_train[idx]] += 1
	            else:
	                neigh_count[self.Y_train[idx]] = 1
	       
	        sorted_neigh_count = sorted(neigh_count.items(), reverse=True)
	        predictions.append(sorted_neigh_count[0][0]) 
	    return predictions


In [9]:
k = 3
start = time.time()
model = KNN(K = k)
model.fit(X_train, y_train)
pred = model.predict(X_test)
acc = accuracy_score(y_test, pred)
end = time.time()
print(f"Time Taken: {end-start} sec")
print("K = "+str(k)+"; Accuracy: "+str(acc))

Time Taken: 6.400968790054321 sec
K = 3; Accuracy: 0.9755555555555555


In [5]:
%timeit -n 10 model.predict(X_test)

5.73 s ± 313 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%lprun -f  model.predict model.predict(X_test)